In [22]:
import urllib
import pandas as pd
import numpy as np
import datetime
import string
import pymysql as mdb

In [23]:
maketables=False

In [24]:
topicsfile="/Users/abramvandergeest/Dropbox/insight_work/topic_list.txt"
f=open(topicsfile)
topics=f.readlines()
f.close()

In [25]:
afile="/Users/abramvandergeest/mysql_insightwiki_auth.txt"
a=open(afile)
passwd=a.readline().rstrip()
a.close()
host='localhost'; user='abram.ghost';db='wikidata'
#con = mdb.connect(host, user, passwd, db)
con = mdb.connect(host, user, passwd, db,port=3307)

#I should also create the  database if it doesn't exist

In [26]:
if maketables==True:
    #Makes topics table
    with con:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS `topics`;")
        str="CREATE TABLE `topics` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_label` VARCHAR(64), `topic_string` VARCHAR(64));"
        cur.execute(str)
    
    #Populate topics table
    with con:
        for topic in topics:
            label="".join(topic.lower().split())
            string=topic.rstrip()
            sql="INSERT INTO `topics` (`topic_label`,`topic_string`) VALUES (%s,%s);"
            cur.execute(sql,(label,string))
        
    #Makes page_views table
    with con:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS `page_views`;")
        str="CREATE TABLE `page_views` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_id` INT, `count` INT, `date` VARCHAR(64));"
        cur.execute(str)    

In [27]:
def get_unzip(d,fo,fo2):
    wikiurl="http://dumps.wikimedia.org/other/pagecounts-raw/%4d/%4d-%02d/"%(d.year,d.year,d.month)
    gzfile="pagecounts-%4d%02d%02d-%02d0000.gz"%(d.year,d.month,d.day,d.hour)
    print gzfile
    fn=wikiurl+gzfile
    urllib.urlretrieve(fn,fo)
    !gunzip -c $fo>$fo2
    return

def csv_to_reduceddf(fo2):
    allwiki=pd.read_csv(fo2,delim_whitespace=True)
    allwiki.columns=['lang','article','views','size']
    enwiki=allwiki[allwiki['lang']=='en']
    enwiki=enwiki[enwiki['size']>=50000]
    return(enwiki)

In [28]:
#creating a dataframe with all english pages of a large size
print "starting"
year=2015
month=1
day=9
hour=0
fo="/Users/abramvandergeest/wikidata/temp.gz"
fo2="/Users/abramvandergeest/wikidata/temp.txt"
d=datetime.datetime(year,month,day,hour)
dend=datetime.datetime(2015,9,12,23)
dend=datetime.datetime(2015,2,28,0)
#dend=datetime.datetime.now()
print "end date is :",dend
#dt=datetime.timedelta(days=+1)
#dt=datetime.timedelta(hours=+1)
dt=datetime.timedelta(hours=+3)

#data is up to date as of:
#2015-09-21 07:41:18.258460


starting
end date is : 2015-02-28 00:00:00


In [29]:
print "entering loop"

while d<datetime.datetime(dend.year,dend.month,dend.day,dend.hour):
    try:
        get_unzip(d,fo,fo2)  #gets the gz file at wikipedia and unzips it
        print "it is unzipped"
        
        enwiki=csv_to_reduceddf(fo2) #takes the unzipped file and pairs it down and puts it into a df
        print "english entries separated"
        
        date='%4d%02d%02d%02d'%(d.year,d.month,d.day,d.hour)
        dic={}
        views=[]
        with con:
            cur = con.cursor()
            sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
            cur.execute(sql)
            for row in cur:
                tops=row[2].rstrip().split()
                twiki=enwiki[np.logical_and.reduce([enwiki['article'].str.contains(top,case=False) for top in tops ])==True]
                t=row[2].rstrip()
                dic[t]=twiki['views'].sum(axis=0)
                #print row[0],dic[t],date
                views.append([row[0],dic[t],date,date[:8],int(date[8:])])
        print "pageviews sorted"
        #print [row[0],dic[t],date,date[:8],int(date[8:])]
        with con:
            cur = con.cursor()
            for view in views:
                sql="INSERT INTO `page_views` (`topic_id`,`count`,`date`,`dateonly`,`timeonly`) VALUES (%s,%s,%s,%s,%s);"
                cur.execute(sql,view)
    
        print date
        d+=dt
    except Exception, e:    
        print "Hit an exception but we are continuing: %s",e
        d+=dt

entering loop
pagecounts-20150109-000000.gz
it is unzipped
english entries separated
pageviews sorted
2015010900
pagecounts-20150109-030000.gz
it is unzipped
english entries separated
pageviews sorted
2015010903
pagecounts-20150109-060000.gz
it is unzipped
english entries separated
pageviews sorted
2015010906
pagecounts-20150109-090000.gz
it is unzipped
english entries separated
pageviews sorted
2015010909
pagecounts-20150109-120000.gz
it is unzipped
english entries separated
pageviews sorted
2015010912
pagecounts-20150109-150000.gz
it is unzipped
english entries separated
pageviews sorted
2015010915
pagecounts-20150109-180000.gz
it is unzipped
english entries separated
pageviews sorted
2015010918
pagecounts-20150109-210000.gz
it is unzipped
english entries separated
pageviews sorted
2015010921
pagecounts-20150110-000000.gz
it is unzipped
english entries separated
pageviews sorted
2015011000
pagecounts-20150110-030000.gz
it is unzipped
english entries separated
pageviews sorted
2015011

KeyboardInterrupt: 